## Using the Jupyter notebook
This is an interactive workbook, where you can run python code. You must have a working python environment with the required libraries installed to use the workbook.

The workbook contains both text cells with instructions (like this cell) and code cells, where you can write code and execute it. Some of the code is already filled in. In that case you can execute the code by clicking on the cell and hit &lt;shift&gt; + &lt;enter&gt;. In the assignments, you will need to fill in some code first yourself.
    
* Use `<Shift> + <Enter>` to execute a cell and advance tot the next cell.
* Use `<Ctrl> + <Enter>` to execute a cell and NOT advance to the next celll.

## ACT API
Most of the functionality in the ACT platform can be accessed through a REST API. Here are some examples using curl:

### Get a list of all object types
`curl -XGET --header 'ACT-User-ID: 1' http://act-eu2.mnemonic.no/v1/objectType`

Example output, using jq and sort to filter/sort the result

```
$ curl -XGET \
  --silent \
  --header 'ACT-User-ID: 1' \
  http://act-eu2.mnemonic.no/v1/objectType \
  | jq '.data[] | .name' \
  | sort
"accountNumber"
"asn"
"campaign"
"certificate"
"credential"
"cryptocurrencyAddress"
"goal"
(...)
```

### Search for facts with object value "127.0.0.1"
`curl -X POST \
  --header 'Content-Type: application/json' \
  --header 'ACT-User-ID: 1' \
  -d '{"objectValue":["127.0.0.1"] }' \
  http://act-eu2.mnemonic.no/v1/fact/search`
  
### Swagger
Swagger is used both for API documentation and for experimenting with the API. If you have access to a private instance (not read only), swagger can be accessed at http://[host]/swagger/.

## Python API
A high level Python API is created to ease the use of the API. You can find the stable version, including documentation on [pypi](https://pypi.org/project/act-api/) and the development version on [github](https://github.com/mnemonic-no/act-api-python).

# Connecting to the API

To connect to the API import the ACT library, and instantiate `act.Act`

In [ ]:
import act

c = act.Act("http://act-eu2.mnemonic.no", user_id = 1, log_level = "warning")

help(c)

## Fact Types / Object Types

With the client you now have access to most of the API, and we can for instance get a list of all Object Types:

In [ ]:
for obj_type in c.get_object_types():
    print(obj_type.name)

## Assignment 1

Write code to get list of all fact types.

Hint: look at the code to list object types above and use the function `get_fact_types()` instead.

In [ ]:
# Write code to print all fact types here



## Facts / Objects

Facts/Objects can be retrived by searching type/value or directly through its id (its unique uuid).

In [ ]:
facts = c.fact_search(object_type="ipv4", object_value="149.202.29.114")
print(facts.size)
one_fact = facts[0]

Above we retrived the first fact in the variable `one_fact`
Values can be obtained using dot notation. Objects related to a fact are stored in the objects array and there can be 1 or two values in this array.

In [ ]:
# Print first fact

print("""
Fact type: {}
Fact value: {}
Object 1 type: {}
Object 1 value: {}
Object 1 direction: {}
Object 2 type: {}
Object 2 value: {}
Object 2 direction: {}
""".format(
  one_fact.type.name,
  one_fact.value,  
  one_fact.objects[0].type.name,  
  one_fact.objects[0].value,  
  one_fact.objects[0].direction,  
  one_fact.objects[1].type.name,  
  one_fact.objects[1].value,  
  one_fact.objects[1].direction,  

))

## Assignment 2

Below a fact is retrieved by its id.. All facts with the same type/value will have the same id.

Q: what is the fact type of this fact?

Q: what are the objects connected to this fact?

In [ ]:
fact = c.fact(id="021cce2f-808c-49e8-957f-dcc992c15cf6").get()

# Enter code to get get fact type and objects for this fact



## Assignment 3

In this scenario we have received logs for a period of suspicious activity. You can retrieve the logs with the code below.

In [ ]:
import json
logs = json.loads(open("logs.json").read())

The log is an array of dictionaries. With the following keys:
* timestamp
* vendor
* product
* signature
* user
* src_ip [optional]
* dest_ip [optional]
* dest_port [optional]
* file_hash [optional]

Execute the code below to see what type of logs (vendor/product) we have.

In [ ]:
set(["{}/{}".format(log["vendor"], log["product"]) for log in logs])

In [ ]:
# Get list of unique IP addresses
dest_ip = list(set([log.get("dest_ip") for log in logs if log.get("dest_ip")]))

In [ ]:
print("Number of IPs: {}".format(len(dest_ip)))

facts = c.fact_search(object_type="ipv4", object_value=dest_ip)
for fact in facts:
    print(fact.objects[0].value, fact.type.name, fact.value, fact.objects[1].value)

In the result above there are different type of facts, and not necessarily all of them are signs of suspicious activity. 

* What type of facts are found?
* Which source IPs / users are involved?
* Which facts are most likely related to suspicious activity?

In [ ]:
# Get source IPs / username of users that are involved


## Other indicators

In the above search, we looked at IP addresses. What other indicators do we have in our logs that can be used to search for in the platform?

### Hint
* Look at available object_types in the act platform and compare with what type of logs we have

In [ ]:
## Extract other indicators from the logs and search the platform for these indicators



## Assignment 4 - Incident

We will no explore how to create facts in the platform. To do this, we will need to connecto to another instance, since act-eu2 is readonly. The password is the sames as given on the PPT slides shown earlier.

In [ ]:
cw = act.Act(
    "http://act-eu3.mnemonic.no", 
    user_id = 1,
    log_level = "warning",
    requests_common_kwargs = {"auth": ("act", "<PASSWORD>")})

We do not create objects in the platform, they are created indirectly whenver we add a 
fact that refer to a object.

So what we need to to is to add facts, and they can be added like this:
    
```
    fact = cw.fact(
         fact_type, fact_value) \
           .source(source_object_type, source_object_value) \
           .destination(destination_object_type, destination_object_value)
```

The code above only creats a local representation of the fact. To add the fact to the platform, it must be added using `add()`:

```
    fact.add()
```

For connecting objects to reports, incidents, etc, we will use the `seenIn` fact. Some facts have validators that only accepts specific values, and we can get these parameters from the API:


In [ ]:
# Show validator parameters for seenIn Fact
[ft.validator_parameter for ft in cw.get_fact_types() if ft.name == "seenIn"][0]

No we want to add the facts from the analysis above.

* Which fact(s) do we want to add?
* Hint: you can decide for yourself the value of an incident, as this is normally an internal reference. A god idea would be to put your name as part of the value in this excersise, for instance 'incident-&lt;yourname&gt;'. 
* Add the fact(s) to the platform

In [ ]:
### Write code to add fact(s)



Open a browser with the link to look at your incident in the GUI:
    
    http://act-eu3.mnemonic.no/object-fact-query/incident/incident-&lt;your-name&gt;
        (replace incident-<your-name> with the value you specified when creating the fact)
    
    Try to clik on [Resolve Facts]. What happens?

## Helper function - `pp()`
The code below will give you some helper functions, where we will use this:
* `pp(result)` - pretty print list or single fact/objects

Use `pp(result, details=False)` to exclude object type and fact values form the result.

In [ ]:
src_obj_direction = {
    "FactIsDestination": " -> ",
    "FactIsSource": " <- ",
    "BiDirectional": " - "
}

dest_obj_direction = {
    "FactIsDestination": " <- ",
    "FactIsSource": " -> ",
    "BiDirectional": " - "
}

def format_obj(obj, details=True):
    out = ""
    
    if details:
        out += "{}:".format(obj.type.name)
    
    out += obj.value
    return out
    

def pobj(obj, details=True):
    print(format_obj(obj, details=details))
    

def pfact(fact, details=True):
    out = ""
    
    src_obj = fact.objects[0]
    
    if len(fact.objects) > 1:
        dest_obj = fact.objects[1]
    else:
        dest_obj = None

    out += format_obj(src_obj, details=details)
    out += src_obj_direction[src_obj.direction]

    out += fact.type.name
    
    if details and not fact.value.startswith("-"):
        out += ":" + fact.value
    
    if dest_obj:
        out += dest_obj_direction[dest_obj.direction]
        out += format_obj(dest_obj, details=details)
        
    print(out)
    
def pp(items, details=True):
    if not isinstance(items, (list, tuple, act.base.ActResultSet)):
        items = [items]
          
    for element in items:
        if isinstance(element, act.obj.Object):
            pobj(element, details=details)
        elif isinstance(element, act.fact.Fact):
            pfact(element, details=details)
        else:
            print(element)

You can now use this method to print the facts:

In [ ]:
facts = c.fact_search(object_type="ipv4", object_value="151.80.241.83")
pp(facts)

## Assignment 5 - Graph queries

Graph queries are implemented in the platform using the [Apache TinkerPop stack](http://tinkerpop.apache.org/). The TinkerPop stack uses a query language, gremlin to support graph traversals. In this assignment we will construct some simple graph queries, using the Python API.

The documentation for Apache TinkerPop can be found here:
* [http://tinkerpop.apache.org/docs/current/](http://tinkerpop.apache.org/docs/current/)
* [http://tinkerpop.apache.org/docs/current/reference/](http://tinkerpop.apache.org/docs/current/reference/)

A more approachable book can be found here:

* [http://kelvinlawrence.net/book/Gremlin-Graph-Guide.html](http://kelvinlawrence.net/book/Gremlin-Graph-Guide.html)

In addtion it is handy to know how object and facts hare mapped to the graph:

* object -> nodes
* facts -> egdes

Graph queries can be executed with the following function in the Python API:

`c.object(type, value).traverse(gremin_query)`

The (useless and) simple query below will return the starting node:

In [ ]:
obj = c.object("ipv4", "149.202.29.77").traverse('g')
pp(obj)

Let us explore more closely the facts associated with this object by stepping in/out from the initial object.

To achive this, we use these gremlin functions:
* `in()` - traverse to other objects pointing to us
* `out()` - traverse to other objects which we point to
* `both()` - travers to other objects either pointing to us or we point to

Try using g.in(), g.out() and g.both() from our starting node.

In [ ]:
obj = c.object("ipv4", "149.202.29.77").traverse('g.in()')
pp(obj)

Q: What do we get here?

Compare with what you see here:

[https://act-eu2.mnemonic.no/object-fact-query/ipv4/149.202.29.77](https://act-eu2.mnemonic.no/object-fact-query/ipv4/149.202.29.77)

Right click on the edges in the graph. 

Q: What are we missing form the result above?

Hint: We have used `in()` to step to the next node, but there also exists a function `inE()`. What is the difference? Try using inE() instead below.

In [ ]:
# Write a query that adds the information missing compared to what you see in the GUI


What other type of C2 infrastructure is involved in the hashes observed using "149.202.29.77" as C2?

In [ ]:
### Write a graph query that finds other type of C2 infrastructure



## Assignment 6 - more advanced graph queries

Scenario: Forensics has been conducted on a machine, and a suspicious sample with sha256 _95b7c2155d7dc60d097bdabbd5c42934a7828d1ae4a5e4b4119c9a07ebc15a96/details_ was found. Is this sample related to the same case, and if so, how?

Hints:
* https://www.virustotal.com/#/file/95b7c2155d7dc60d097bdabbd5c42934a7828d1ae4a5e4b4119c9a07ebc15a96
* graph via tool or campaign
* bothE().otherV()
    

In [ ]:
# Write a graph query finds any link through tool or campaign to the the other finding

